# 5.4.3 可视化类激活的热力图

类激活图可视化（CAM）
表示每个位置的重要程度，比如输入猫的图像，生成类别“猫”的热力图，表示图像的各个部分与猫的相似程度

用每个通道对类别的重要程度，对输入图像对不同通道的激活强度的空间图进行加权，从而得到输入图像对类别的激活强度

注意这次我们并没有舍弃掉最后的全连接层

In [1]:
from keras.applications.vgg16 import VGG16
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
model = VGG16(weights='imagenet')

2021-12-13 22:11:44.437631: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-13 22:11:44.979419: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9063 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


利用内置函数，处理为VGG16模型能够读取的格式

In [2]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input,decode_predictions
import numpy as np

img_path='/home/wufisher/dataset_m/h5inside/elephant.jpg'

img = image.load_img(img_path,target_size=(224,224))

x=image.img_to_array(img)

x=np.expand_dims(x,axis=0)

x=preprocess_input(x)

preds=model.predict(x)
print('Rredicted:',decode_predictions(preds,top=3)[0])

2021-12-13 22:11:47.280506: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2021-12-13 22:11:47.835486: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


Rredicted: [('n02504458', 'African_elephant', 0.9094216), ('n01871265', 'tusker', 0.086182274), ('n02504013', 'Indian_elephant', 0.0043545687)]


打印一下最大对应激活元素非洲象的索引

In [3]:
print(np.argmax(preds[0]))

386


展现哪些部分最像非洲象，利用gan算法，并且将热力图标准化到01范围

In [4]:
# with tf.GradientTape() as gtape:
#     grads = gtape.gradient(african_elephant_output, last_conv_layer.output)

In [5]:
# from tensorflow.keras import backend as K
from tensorflow.compat.v1.keras import backend as K
import tensorflow as tf
##注意修改递归深度
tf.compat.v1.disable_eager_execution()
# tf.compat.v1.disable_v2_behavior()
import sys  # 导入sys模块
sys.setrecursionlimit(30000)  # 将默认的递归深度修改为3000
# print(model.output[:,386])
african_elephant_output = model.output[:,386]   # 预测向量中的非洲象元素

last_conv_layer = model.get_layer('block5_conv3')  # block5_conv3层的输出特征图，它是VGG16的最后一个卷积层

grads = K.gradients(african_elephant_output, last_conv_layer.output)[0]   # 非洲象类别相对于block5_conv3输出特征图的梯度
# with tf.GradientTape() as gtape:
#      grads = gtape.gradient(african_elephant_output, last_conv_layer.output)
pooled_grads = K.mean(grads, axis=(0, 1, 2))   # 形状是（512， ）的向量，每个元素是特定特征图通道的梯度平均大小

iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])  # 这个函数允许我们获取刚刚定义量的值：对于给定样本图像，pooled_grads和block5_conv3层的输出特征图

pooled_grads_value, conv_layer_output_value = iterate([x])  # 给我们两个大象样本图像，这两个量都是Numpy数组

for i in range(512):
     conv_layer_output_value[:, :, i] *= pooled_grads_value[i]  # 将特征图数组的每个通道乘以这个通道对大象类别重要程度

heatmap = np.mean(conv_layer_output_value, axis=-1)  # 得到的特征图的逐通道的平均值即为类激活的热力图

In [ ]:
import matplotlib.pyplot as plt

# 之前v6的图太多了
plt.clf()
heatmap = np.maximum(heatmap,0)
heatmap /=np.max(heatmap)
#print(heatmap)
plt.matshow(heatmap)

Error: Session cannot generate requests

In [ ]:
import cv2
img = cv2.imread(img_path)
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
# 将热力图应用于原始图像
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
# 这里的 0.4 是热力图强度因子
superimposed_img = heatmap * 0.4 + img
cv2.imwrite('elephant_cam_last.jpg', superimposed_img)

Error: Session cannot generate requests